In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
url = requests.get('https://top14.lnr.fr/statistiques/joueurs')
soup = BeautifulSoup(url.text, 'html.parser')

In [3]:
url

<Response [200]>

In [4]:
# print(url.text)

In [5]:
# soup

In [6]:
# # FONCTIONNE
# # RECUPERATION DES TITRES DES DE CHAQUE STATS AINSI QUE LES 3 MEILLEURS JOUEURS

# soup.find_all('div', {'class': 'season-top-block__wrapper'})

# # 1ere boucle
# for title in soup.find_all('div', {'class': 'season-top-block__wrapper'}):
#     print(title.find_all('h3')[0].get_text())

# # 2 eme boucle    
#     for player in range(len(ele.find_all('a', {'class':'season-top-line__name'}))):
#         print(title.find_all('a', {'class':'season-top-line__name'})[player].get_text().strip())


# a transformer en fonction

In [7]:
list_annee = ['2019-2020', '2020-2021', '2021-2022', '2022-2023', '2023-2024', '2024-2025']

In [8]:
dico_category = {0:"Meilleurs réalisateurs",
                 1:"Meilleurs marqueurs d\'essais",
                 2:"Meilleurs marqueurs de drops",
                 3:"Meilleurs taux de transformation",
                 4:"Plus de temps de jeu",
                 5:"Moyene de points par match la plus haute",
                 6:"Nombre de cartons"}

df = pd.DataFrame()
stats_total = []
df_final = pd.DataFrame()

for annee in list_annee:

    url = requests.get(f'https://top14.lnr.fr/statistiques/joueurs/{annee}?')
    soup = BeautifulSoup(url.text, 'html.parser')

    # Initialiser des listes vides pour chaque catégorie
    meilleurs_real = []
    meilleurs_marqueur_essais = []
    meilleurs_marqueur_drops = []
    meilleurs_taux_transfo = []
    temps_de_jeu = []
    moyenne_points_match = []
    nombre_cartons = []

    # je recherche via les balises les informations
    for title in soup.find_all('div', {'class': 'season-top-block__wrapper'}):
        # je recherche tous les titres des catégories
        # le titre correspond à chaque catégorie recherchée
        titre = title.find('h3').get_text().strip()
        
        # récupération des joueurs popur chaque liste
        # pour chaque joueurs depuis ma soup via find_all (balise et class), je récupère leurs noms avec get_text() et .strip() pour supprimer les espaces
        player = [player.get_text().strip() for player in title.find_all('a', {'class': 'season-top-line__name'})]
        stats = [stats.get_text().strip() for stats in title.find_all('div', {'class': 'season-top-line__stat-number'})]
        stats_total = stats_total + stats
        # print(stats_total)
        # associer chaque joueurs à la liste correspondantes avec des conditions 
        if titre == "Meilleur réalisateur":
            meilleurs_real = player
        elif titre == "Meilleur marqueur d'essais":
            meilleurs_marqueur_essais = player
        elif titre == "Meilleur marqueur de drops":
            meilleurs_marqueur_drops = player
        elif titre == "Meilleur taux de transfo":
            meilleurs_taux_transfo = player
        elif titre == "Temps de jeu":
            temps_de_jeu = player
        elif titre == "Moyenne de points par match":
            moyenne_points_match = player
        elif titre == "Nombre de cartons":
            nombre_cartons = player
    
    all_list = [meilleurs_real, meilleurs_marqueur_essais, meilleurs_marqueur_drops, meilleurs_taux_transfo, temps_de_jeu, moyenne_points_match, nombre_cartons]

    for list in all_list:
        list[0] = list[0].replace('\xa0', "")
        # print(list)

    for indice, list in enumerate(all_list):

        df_stats = pd.DataFrame({
            "RANK":[1, 2, 3],
            "CATEGORY":dico_category[indice],
            "NAME":list,
            "SEASON":annee
        })
        
        df = pd.concat([df, df_stats])
        
df['STATS'] = stats_total

df_final = pd.concat([df_final, df])

df_final.reset_index(drop = True, inplace = True)

In [9]:
df_final

,RANK,CATEGORY,NAME,SEASON,STATS
0,1,Meilleurs réalisateurs,T.LARANJEIRA,2019-2020,176
1,2,Meilleurs réalisateurs,Anthony BELLEAU,2019-2020,171
2,3,Meilleurs réalisateurs,Antoine HASTOY,2019-2020,139
3,1,Meilleurs marqueurs d'essais,A.RETIERE,2019-2020,8
4,2,Meilleurs marqueurs d'essais,Thomas COMBEZOU,2019-2020,7
...,...,...,...,...,...
121,2,Moyene de points par match la plus haute,Nolann LE GARREC,2024-2025,"12,1"
122,3,Moyene de points par match la plus haute,Léo BERDEU,2024-2025,"11,5"
123,1,Nombre de cartons,C.WOKI,2024-2025,3
124,2,Nombre de cartons,Apisai NAQALEVU,2024-2025,3


In [10]:
len(stats_total)

126

In [11]:
df = pd.DataFrame()

for indice, list in enumerate(all_list):

    df_stats = pd.DataFrame({
        "RANK":[1, 2, 3],
        "CATEGORY":dico_category[indice],
        "NAME":list 
    })
    
    df = pd.concat([df, df_stats])
    
df

    

,RANK,CATEGORY,NAME
0,1,Meilleurs réalisateurs,J.SIMMONDS
1,2,Meilleurs réalisateurs,Léo BERDEU
2,3,Meilleurs réalisateurs,Maxime LAFAGE
0,1,Meilleurs marqueurs d'essais,G.DREAN
1,2,Meilleurs marqueurs d'essais,Sireli MAQALA
2,3,Meilleurs marqueurs d'essais,Baptiste COUILLOUD
0,1,Meilleurs marqueurs de drops,J.SIMMONDS
1,2,Meilleurs marqueurs de drops,Maxime LAFAGE
2,3,Meilleurs marqueurs de drops,Louis CARBONEL
0,1,Meilleurs taux de transformation,J.MALLIA


In [12]:
df.to_csv("df_stats_jouerurs_top_3.csv")

# Nouveau système avec avancement année par année

In [13]:
# print(meilleurs_real)
# print(meilleurs_marqueur_essais)
# print(meilleurs_marqueur_drops)
# print(meilleurs_taux_transfo)
# print(temps_de_jeu)
# print(moyenne_points_match)
# print(nombre_cartons)

In [14]:
# meilleurs_real[0].replace('\xa0', "")

In [15]:
# all_list = [meilleurs_real, meilleurs_marqueur_essais, meilleurs_marqueur_drops, meilleurs_taux_transfo, temps_de_jeu, moyenne_points_match, nombre_cartons]

# for list in all_list:
#     list[0] = list[0].replace('\xa0', "")
#     print(list)

# CREATION DE LA FONCTION TOP 3

In [16]:
def fn_top_3():
    # Initialiser des listes vides pour chaque catégorie
    meilleurs_real = []
    meilleurs_marqueur_essais = []
    meilleurs_marqueur_drops = []
    meilleurs_taux_transfo = []
    temps_de_jeu = []
    moyenne_points_match = []
    nombre_cartons = []

    # je recherche via les balises les informations
    for title in soup.find_all('div', {'class': 'season-top-block__wrapper'}):
        # je recherche tous les titres des catégories
        # le titre correspond à chaque catégorie recherchée
        titre = title.find('h3').get_text().strip()
        
        # récupération des joueurs popur chaque liste
        # pour chaque joueurs depuis ma soup via find_all (balise et class), je récupère leurs noms avec get_text() et .strip() pour supprimer les espaces
        player = [player.get_text().strip() for player in title.find_all('a', {'class': 'season-top-line__name'})]

        # associer chaque joueurs à la liste correspondantes avec des conditions 
        if titre == "Meilleur réalisateur":
            meilleurs_real = player
        elif titre == "Meilleur marqueur d'essais":
            meilleurs_marqueur_essais = player
        elif titre == "Meilleur marqueur de drops":
            meilleurs_marqueur_drops = player
        elif titre == "Meilleur taux de transfo":
            meilleurs_taux_transfo = player
        elif titre == "Temps de jeu":
            temps_de_jeu = player
        elif titre == "Moyenne de points par match":
            moyenne_points_match = player
        elif titre == "Nombre de cartons":
            nombre_cartons = player

    all_list = [meilleurs_real, meilleurs_marqueur_essais, meilleurs_marqueur_drops, meilleurs_taux_transfo, temps_de_jeu, moyenne_points_match, nombre_cartons]

    for list in all_list:
        list[0] = list[0].replace('\xa0', "")
        print(list)


In [17]:
fn_top_3()

['J.SIMMONDS', 'Léo BERDEU', 'Maxime LAFAGE']
['G.DREAN', 'Sireli MAQALA', 'Baptiste COUILLOUD']
['J.SIMMONDS', 'Maxime LAFAGE', 'Louis CARBONEL']
['J.MALLIA', 'Maxime LUCU', 'Thibault DAUBAGNA']
['F.GORRISSEN', 'Joe SIMMONDS', 'Thomas CEYTE']
['T.RAMOS', 'Nolann LE GARREC', 'Léo BERDEU']
['C.WOKI', 'Apisai NAQALEVU', 'Levan CHILACHAVA']
